In [1]:
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import fitsio as fio
import numpy as np
import ast

%run /home/cz136/project/sa/lib/header.ipynb

In [2]:
cluster_table=Table(fits.open('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17-vlim_lgt20_desformat_catalog.fit')[1].data)
member_table=Table(fits.open('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17-vlim_lgt20_desformat_catalog_members.fit')[1].data)

cluster_table.remove_columns(['P_C','C_LAMBDA','C_LAMBDA_ERR','Q_CEN']) #unknown columns

print(len(member_table))
print(len(cluster_table))

399002
7066


In [3]:
mcal = fio.FITS('/lsst/troxel/y1a1/mcal-y1a1-combined-riz-unblind-v4-matched.fits',memmap=True,mode='r')[1]
print ("Metacal loaded")
shape_id=mcal['coadd_objects_id'][:]

Metacal loaded


In [4]:
problem_list =['MODEL_MAG', 'MODEL_MAGERR', 'MAG_LAMBDA_ERR', 'PZBINS',
               'PZ', 'RA_CENT', 'DEC_CENT', 'ID_CENT', 'LAMBDA_CHISQ_CENT',
               'ZLAMBDA_CENT', 'P_BCG', 'P_CEN', 'P_FG', 'P_SAT']

In [5]:
band_col_list = []
pz_col_list = []
alt_col_list =[]
for problem in problem_list:
    if cluster_table[problem].shape[1]==4:
        band_col_list.append(problem)
    elif cluster_table[problem].shape[1]==5:
        alt_col_list.append(problem)
    elif cluster_table[problem].shape[1]==21:
        pz_col_list.append(problem)
    else:
        print ("error")

In [6]:
band_list=['g','r','i','z','Y']
pz_index_list=['pz'+str(i+1) for i in range(21)]
alt_index_list=['Alt'+str(i+1) for i in range(5)]

In [7]:
for j in range(4):
    for i,column in enumerate(band_col_list):
        cluster_table[str(('Band',band_list[j],column))]=cluster_table[column].data[:,j]
cluster_table.remove_columns(band_col_list)

In [8]:
for i,column in enumerate(pz_col_list):
    for j in range(21):
        cluster_table[str(('PZ',pz_index_list[j],column))]=cluster_table[column].data[:,j]
cluster_table.remove_columns(pz_col_list)

In [9]:
for i,column in enumerate(alt_col_list):
    for j in range(5):
        cluster_table[str(('Alt',alt_index_list[j],column))]=cluster_table[column].data[:,j]
cluster_table.remove_columns(alt_col_list)

In [10]:
all_col_list=list(cluster_table.columns[0:54])
for i,column in enumerate(all_col_list):
    cluster_table[str(('All','All',column))]=cluster_table[column].data[:]
cluster_table.remove_columns(all_col_list)

In [11]:
for j in range(4):
    for i,column in enumerate(band_col_list[0:2]):
        member_table['('+band_list[j]+','+column+')']=member_table[column].data[:,j]
member_table.remove_columns(band_col_list[0:2]) 

In [12]:
clusters=cluster_table.to_pandas()
member=member_table.to_pandas()
print ("Center& member loaded")
# center_table=None
# member_table=None
print ("Memory cleared")

Center& member loaded
Memory cleared


In [13]:
member_cols = member.columns[0:22]
shape_columns=pd.MultiIndex.from_product([['All'],list(mcal.get_colnames())],names=['Band','Data'])

In [14]:
center_bins=[m+str(n) for m,n in zip(['cen_']*5,np.arange(5)+1)]
center=pd.MultiIndex.from_product([['Center'],center_bins,member_cols])

In [15]:
cluster_band_columns=pd.MultiIndex.from_product([['g','r','i','z'],
                                       ['MODEL_MAG','MODEL_MAGERR','MAG_LAMBDA_ERR']],
                                     names=['band','data'])

In [16]:
# alt_bins=[m+str(n) for m,n in zip(['alt_']*5,np.arange(5)+1)]
# alt_columns=pd.MultiIndex.from_product([alt_bins,
#                                        alt_col_list.append(shape_names)],
#                                      names=['alt_index','data'])

In [17]:
member_band_columns=pd.MultiIndex.from_product([['g','r','i','z'],
                                       ['MODEL_MAG','MODEL_MAGERR',]],
                                     names=['Band','Data'])

In [18]:
member_band_columns

MultiIndex([('g',    'MODEL_MAG'),
            ('g', 'MODEL_MAGERR'),
            ('r',    'MODEL_MAG'),
            ('r', 'MODEL_MAGERR'),
            ('i',    'MODEL_MAG'),
            ('i', 'MODEL_MAGERR'),
            ('z',    'MODEL_MAG'),
            ('z', 'MODEL_MAGERR')],
           names=['Band', 'Data'])

In [19]:
shape_index=pd.MultiIndex.from_tuples([('MEM_MATCH_ID','TYPE')],names=['ClusterID','Type'])

In [20]:
shape_df=pd.DataFrame(index=shape_index,columns=member_band_columns)

In [21]:
shape_df.fillna(value=1)

Band                      g                      r                      i  \
Data              MODEL_MAG MODEL_MAGERR MODEL_MAG MODEL_MAGERR MODEL_MAG   
ClusterID    Type                                                           
MEM_MATCH_ID TYPE         1            1         1            1         1   

Band                                   z               
Data              MODEL_MAGERR MODEL_MAG MODEL_MAGERR  
ClusterID    Type                                      
MEM_MATCH_ID TYPE            1         1            1

In [22]:
mcal_df =pd.DataFrame(index=shape_index,columns=shape_columns) 

In [23]:
mcal_df.fillna(value=1)

Band                           All                                         \
Data              coadd_objects_id flags mask_frac psf_e1 psf_e2 psf_size   
ClusterID    Type                                                           
MEM_MATCH_ID TYPE                1     1         1      1      1        1   

Band                                                        ...                \
Data              mcal_psf_e1 mcal_psf_e2 mcal_psf_size e1  ... R22 region ra   
ClusterID    Type                                           ...                 
MEM_MATCH_ID TYPE           1           1             1  1  ...   1      1  1   

Band                                                                         \
Data              dec tilename flags_select flags_select_1p flags_select_1m   
ClusterID    Type                                                             
MEM_MATCH_ID TYPE   1        1            1               1               1   

Band                                               
Data              flags_select_2p flags_select_2m  
ClusterID    Type                                  
MEM_MATCH_ID TYPE               1               1  

[1 rows x 80 columns]

In [24]:
pd.concat([mcal_df,shape_df],axis=1)

Band                           All                                         \
Data              coadd_objects_id flags mask_frac psf_e1 psf_e2 psf_size   
ClusterID    Type                                                           
MEM_MATCH_ID TYPE              NaN   NaN       NaN    NaN    NaN      NaN   

Band                                                          ...  \
Data              mcal_psf_e1 mcal_psf_e2 mcal_psf_size   e1  ...   
ClusterID    Type                                             ...   
MEM_MATCH_ID TYPE         NaN         NaN           NaN  NaN  ...   

Band                                                      g               \
Data              flags_select_2p flags_select_2m MODEL_MAG MODEL_MAGERR   
ClusterID    Type                                                          
MEM_MATCH_ID TYPE             NaN             NaN       NaN          NaN   

Band                      r                      i                      z  \
Data              MODEL_MAG MODEL_MAGERR MODEL_MAG MODEL_MAGERR MODEL_MAG   
ClusterID    Type                                                           
MEM_MATCH_ID TYPE       NaN          NaN       NaN          NaN       NaN   

Band                            
Data              MODEL_MAGERR  
ClusterID    Type               
MEM_MATCH_ID TYPE          NaN  

[1 rows x 88 columns]

In [25]:
member_id=member['ID'].values

In [26]:
# for id in member_id:
#     mask = pd.Series(shape_id)==id

In [27]:
shape_id_series = pd.Series(shape_id)

In [28]:
iloc=pd.Series(np.arange(len(shape_id_series)))

In [29]:
temp=pd.concat([shape_id_series,iloc],axis=1)

In [30]:
def sort2n(x,y):
    """
    Sorts and matches two arrays of object ids where x is unique and y is not (in DES this is coadd_objects_id).
    Slower than sort2().
    """
    
    xsort = np.argsort(x)
    ysort = np.argsort(y)
    i_yx = np.sort(y[np.in1d(y, x, assume_unique=False)])
    i_x = xsort[x[xsort].searchsorted(i_yx)]
    i_y = ysort[y[ysort].searchsorted(i_yx)]
    
    return i_x, i_y

In [31]:
s2n_index=sort2n(member_id,shape_id)

In [32]:
s2n_index[1]

array([       64,        82,       201, ..., 134632602, 134632624,
       134632632])

In [33]:
shape_half=pd.DataFrame(mcal.read(rows=[s2n_index[1]]).byteswap().newbyteorder())

In [34]:
member_half=member.iloc[s2n_index[0]]

In [35]:
member_half_index=pd.MultiIndex.from_tuples(list(zip(['All']*23,member_half.columns[0:23]))+list(member_band_columns),names=['Band','Data']) 

In [36]:
shape_half.columns=shape_columns
member_half.columns=member_half_index
shape_half.set_index(('All','coadd_objects_id'),inplace=True)
member_half.set_index(('All','ID'),inplace=True)

In [37]:
shape_df=pd.concat([member_half,shape_half],axis=1,verify_integrity=True)
shape_df.index.name="ID"

In [38]:
cluster_col_list=[ast.literal_eval(name) for i,name in enumerate(clusters.columns)]
cluster_col=pd.MultiIndex.from_tuples(cluster_col_list, names=['Type','Kind','Data'])

In [39]:
clusters.columns=cluster_col
clusters.set_index(('All', 'All', 'MEM_MATCH_ID'),inplace=True);

In [40]:
def get_theta_for_obj(obj1,obj2):
    ra1,dec1=(obj1[('All', 'RA')],obj1[('All', 'DEC')])
    ra2,dec2=(obj2[('All', 'RA')],obj2[('All', 'DEC')])
    
    
    
    c1=SkyCoord(ra1*u.deg,dec1*u.deg)
    c2=SkyCoord(ra2*u.deg,dec2*u.deg)
    
    
    return(90-c1.position_angle(c2).degree)

In [41]:
def get_pa(member):
    cluster=clusters.loc[member[('All','MEM_MATCH_ID')]]
    center=shapes.loc[cluster[('Alt','Alt1','ID_CENT')]]
    theta=get_theta_for_obj(center,member)
    return(theta)

In [42]:
shape_df[('All','PP')]=shape_df.apply(lambda x:x[('All','P')]*x[('All','PFREE')],axis=1)
shape_df[('All','PP')]=shape_df.apply(lambda x:x[('All','P')]*x[('All','PFREE')],axis=1)
shape_df[('All','Theta_cen')]=shape_df.apply(lambda x:get_pa(x),axis=1)

In [43]:
shape_df[('All','Theta_cen')]=shape_df.apply(lambda x:360-x[('All','Theta_cen')] if x[('All','Theta_cen')]>=180 else x[('All','Theta_cen')],axis=1)
shape_df[('All','Theta_cen')]=shape_df.apply(lambda x:360+x[('All','Theta_cen')] if x[('All','Theta_cen')]<=-180 else x[('All','Theta_cen')],axis=1)
shape_df[('All','Theta_sat')]=shape_df.apply(lambda x:-(180-x[('All','Theta_cen')]) if x[('All','Theta_cen')]>=0 else (180+x[('All','Theta_cen')]),axis=1)

# Calculate cluster elipticity

In [44]:
def drop_poor_centers():
    center_id=clusters[('Alt', 'Alt1', 'ID_CENT')]
    pp=shapes.loc[center_id][('All','PP')].to_numpy()
    missing_mem_match_id=clusters[np.isnan(pp)].index
    clusters.drop(missing_mem_match_id,inplace=True)
    np.save("/home/cz136/project/sa/data/missing_centers.npy",center_id[np.isnan(pp)].to_numpy())
    print("The number of clusters with no center shape data {}".format(np.sum(np.isnan(pp))))
    print("The number of clusters with low probability center {}".format(np.sum(pp[~np.isnan(pp)]<=0.55)))
drop_poor_centers()

The number of clusters with no center shape data 65
The number of clusters with low probability center 3


/home/cz136/anaconda3/envs/balrog/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  This is separate from the ipykernel package so we can avoid doing imports until


In [45]:
display(Math(r"$M_{\mathrm{xx}} \equiv\left\langle\frac{x^{2}}{r^{2}}\right\rangle=\frac{\sum_{i} p_{\mathrm{mem}, i} \frac{x_{i}^{2}}{r_{i}^{2}}}{\sum_{i} p_{\mathrm{mem}, i}}$"))

<IPython.core.display.Math object>

In [46]:
def get_Q_U(cluster):
    
    mem_match_id=cluster.name
    members=shapes[shapes['All','MEM_MATCH_ID']==mem_match_id]
    
    members=members[members[('All','PP')]>=0.2]
    
    def get_M_for_cluster():
        p = members[('All','PP')]
        r = members[('All','R')]
        pa = members[('All','Theta_cen')]
        x = r * np.cos(pa)
        y = r * np.sin(pa)
        
        def mom_2(p,x,y,r):
            return(np.sum(p*x*y/r**2)/np.sum(p))
        
        Mxx=mom_2(p,x,x,r)
        Mxy=mom_2(p,x,y,r)
        Myy=mom_2(p,y,y,r)
        return(Mxx,Mxy,Myy)

    Mxx,Mxy,Myy=get_M_for_cluster()
    Q,U=Mxx-Myy,2*Mxy
    
    return (Q,U)


def get_pa_from_Q_U(Q,U):
    α =0.5*np.arctan2(U,Q)*180/np.pi
    return(α)

In [47]:
clusters[('All','All','Q')]=clusters.apply(lambda x:get_Q_U(x)[0],axis=1)
clusters[('All','All','U')]=clusters.apply(lambda x:get_Q_U(x)[1],axis=1)
clusters[('All','All','Beta')]=clusters.apply(lambda x:get_pa_from_Q_U(x[('All','All','Q')],x[('All','All','U')]),axis=1)
clusters[('All','All','Ce')]=clusters.apply(lambda x:np.sqrt(x[('All','All','U')]**2+x[('All','All','U')]**2),axis=1)

In [48]:
clusters.to_pickle('/home/cz136/project/sa/data/clusters.pkl')

In [49]:
shape_df=shape_df[shape_df[('All', 'flags_select')]==0]

In [50]:
shape_df.to_pickle('/home/cz136/project/sa/data/shapes.pkl')

In [51]:
missing_centers=np.load('/home/cz136/project/sa/data/missing_centers.npy')

In [52]:
np.sum(np.isnan(clusters[('All','All','Ce')].to_numpy()))

0

In [53]:
for ID in missing_centers:
    if np.sum(shape_id==ID):
        print("The missing is wrong!")